<a href="https://colab.research.google.com/github/xelav/Insider-Threat-CERT/blob/master/notebooks/lstm_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing

In [1]:
from pathlib import Path
from google.colab import drive, files
drive.mount('/content/drive')

drive_dir = Path('/content/drive/My Drive/')
root_dir = Path('/content')

assert drive_dir.is_dir()
assert root_dir.is_dir()

%cd "{root_dir}"

Mounted at /content/drive
/content


In [2]:
repo_dir = drive_dir / 'CERT'
assert repo_dir.is_dir()

import sys
sys.path.append(str(repo_dir))

In [3]:
# repo_dir = "/content/repo"
# !git clone https://github.com/xelav/Insider-Threat-CERT "$repo_dir"

# import sys
# sys.path.append(repo_dir)

# repo_branch = 'master'
# %cd "{repo_dir}"
# !git checkout $repo_branch
# !git fetch
# !git reset origin/$repo_branch --hard
# %cd "{root_dir}"

In [4]:
!pip install pytorch-ignite
!pip install ipdb
%pip install wandb==0.8.18 -q

import wandb
wandb.login()
wandb.__version__

     |████████████████████████████████| 204kB 16.6MB/s 
     |████████████████████████████████| 788kB 39.4MB/s 
     |████████████████████████████████| 368kB 49.7MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.7-cp37-none-any.whl size=11434 sha256=746b18f66cf176c5df9ac4ff536445eea92721198947a4dbe32b5ea5b9fc4933
  Stored in directory: /root/.cache/pip/wheels/69/e2/66/bde554f8029ad1c5288f3bf427a78b18ec670182d7e670efe6
Successfully built ipdb
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.21.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


     |████████████████████████████████| 1.3MB 22.7MB/s 
     |████████████████████████████████| 133kB 58.0MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 163kB 35.5MB/s 
     |████████████████████████████████| 256kB 55.2MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 184kB 41.6MB/s 


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'0.8.18'

In [5]:
# branch_name = 'master'

# %cd "{repo_dir}"
# !git reset --hard
# !git checkout new-cnn-parameters
# !git fetch
# !git reset origin/new-cnn-parameters --hard
# %cd "{root_dir}"

In [6]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
from tqdm import tqdm
import gc

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder, LSTM_Encoder_Topics, LSTM_Encoder_Numeric
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.lstm_trainer import *

# Prepare run config



In [ ]:
%ls "{drive_dir}"

 asdf                           notebooks/
'Book Rewards.gsheet'           NY_feasting_supplies.gdoc
 Books/                         private_key.ppk
 CERT/                          runs/
'Colab Notebooks'/              scan2020_10_29_14_58_52.bmp_2@
 Coursera/                     'Untitled Diagram.drawio'
 Datasets/                      wandb/
 df_final.pkl                  'Work Docs'/
 Docs/                         'Новый документ.gdoc'
 edu-cmc-skmodel20-620-01.pub  'План работ.gdoc'
 expert.drawio                 'Результаты моделей-2020.gdoc'
 intermediate.drawio           'Результаты моделей.gdoc'
 masked_lm/                    'Телеком - документация'/
 masked_lm_enc/                 Финансы.gsheet
 MSU/


In [8]:
dataset_version = '6.2'

In [9]:
output_dir = Path(f'{drive_dir}/Datasets/CERT_output_v{dataset_version}')
answers_dir = Path(f"{drive_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

assert(output_dir.is_dir())
assert(answers_dir.is_dir())

# log_dir = output_dir / 'logs' / run_name
# checkpoint_dir = output_dir / 'checkpoints' / run_name

# run_name = 'lstm/content-test7'

# assert(not log_dir.is_dir())
# assert(not checkpoint_dir.is_dir())

# if log_dir.is_dir():
#     shutil.rmtree(log_dir)
# if checkpoint_dir.is_dir():
#     shutil.rmtree(checkpoint_dir)

In [10]:
# monkeypatch symlinks. For some reason, colab doesn't allow to make symlinks on some files
# No idea why this happens, so i did this. It's bad, but it's works.
import os
from shutil import copyfile
os.symlink = copyfile

In [11]:
params = get_params()
params['model']['lstm_encoder']['embedding_size'] = None
params['model']['lstm_encoder']['use_content_topics'] = False
params['model']['lstm_encoder']['condition_vec_size'] = None

# This is a mess of the code..

# add content topics
# WARNING: don't mix embeddings with content topics!
# params['model']['lstm_encoder']['input_size']

In [12]:
# actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl',
#                                                main_answers_file,
#                                                min_length=50,
#                                                max_length=200,
#                                                )

In [13]:
df = pd.read_parquet(output_dir / 'final_df.parquet')

enable_conditional_vecs = False
if enable_conditional_vecs:
    conditional_vecs = df.drop(['action_id', 'malicious', 'user', 'period'], axis=1).values
    params['model']['lstm_encoder']['condition_vec_size'] = conditional_vecs.shape[1]
else:
    conditional_vecs = None

print("Padding_actions")
df.action_id = CertDataset.pad_to_length_numpy(df.action_id, max_length=200)
actions = df.action_id.values
targets = df['malicious']
del df 
import gc
gc.collect()
actions = np.vstack(actions)

params['model']['lstm_encoder']['use_content_topics'] = False
content = None
if params['model']['lstm_encoder']['use_content_topics']:
    content_values = df.content.values
    content_values = CertDataset.pad_topic_matricies(content_values, max_length=200)
    content = df.content

cert_dataset = CertDataset(actions, targets,
                           content_topics=None,
                           conditional_vecs=conditional_vecs)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0, batch_size=1024)

device = 'cuda'

# del actions, reduced_labels, ukeys, index, values, arrays
# gc.collect()

Padding_actions


# Prepare trainers

In [14]:
del params['model']['cnn_classifier']
del params['train']['cnn_classifier']
# params['data_granularity'] = GRANULARITY

In [15]:
wandb.init(project=f"cert{dataset_version}-lstm", config=params)
checkpoint_dir = Path(wandb.run.dir) / 'checkpoints'

if params['model']['lstm_encoder']['use_content_topics']:
    LSTM_model = LSTM_Encoder_Topics
elif params['model']['lstm_encoder']['condition_vec_size']:
    LSTM_model = LSTM_Encoder_Numeric
else:
    LSTM_model = LSTM_Encoder
lstm_encoder = LSTM_model(params['model']['lstm_encoder'])
criterion = nn.NLLLoss()
optimizer = optim.Adam(lstm_encoder.parameters())

train_engine = create_supervised_trainer_lstm(
                                        lstm_encoder, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch_lstm,
                                        checkpoint_dir=checkpoint_dir,
                                       )

val_engine = create_supervised_evaluator_lstm(
        lstm_encoder, device=device,
        prepare_batch=prepare_batch_lstm,
        metrics={},
        criterion=criterion,
        checkpoint_dir=checkpoint_dir,
)

@train_engine.on(Events.STARTED)
def log_training_results(trainer):
    print('Initial validation run:')
    val_engine.train_epoch = 0
    val_engine.run(val_loader)

    wandb.run.summary["best_val_non_pad_accuracy"] = val_engine.state.metrics['non_pad_accuracy']
    wandb.run.summary["best_val_loss"] = val_engine.state.metrics['epoch_loss']

@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    val_engine.train_epoch = train_engine.state.epoch
    val_engine.run(val_loader)

    if val_engine.state.metrics['non_pad_accuracy'] > wandb.run.summary["best_val_non_pad_accuracy"]:
        wandb.run.summary["val_non_pad_accuracy"] = val_engine.state.metrics['non_pad_accuracy']
        wandb.run.summary["best_val_loss"] = val_engine.state.metrics['epoch_loss']
        wandb.run.summary["best_train_non_pad_accuracy"] = train_engine.state.metrics['non_pad_accuracy']
        wandb.run.summary["best_train_loss"] = train_engine.state.metrics['epoch_loss']

        wandb.save(str(checkpoint_dir) + '/best_model*')


wandb: Wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Load word2vec embeddings

In [16]:
if params['model']['lstm_encoder']['embedding_size']:
    pretrained_path = output_dir / 'checkpoints' / 'w2v/test2/final_model_7758.pth'

    pretrained_embeddings = torch.load(pretrained_path)
    pretrained_embeddings = pretrained_embeddings['embed.weight']

    lstm_encoder.embedding = lstm_encoder.embedding.from_pretrained(pretrained_embeddings[:-1])
    lstm_encoder.embedding.requires_grad = False

# Run experiment

In [17]:
params['train']['lstm_encoder']

{'batch_size': 1024, 'lr': 0.001, 'manual_seed': 0, 'num_epochs': 200}

In [ ]:
torch.manual_seed(params['train']['lstm_encoder']['manual_seed'])
train_engine.run(train_loader,
                 max_epochs=params['train']['lstm_encoder']['num_epochs'],
                 seed=params['train']['lstm_encoder']['manual_seed']
                 )
print('finished!')

/usr/local/lib/python3.7/dist-packages/ignite/engine/engine.py:656: UserWarning: Argument seed is deprecated. It will be removed in 0.5.0. Please, use torch.manual_seed or ignite.utils.manual_seed
  "Argument seed is deprecated. It will be removed in 0.5.0. "


Initial validation run:
Validation Results   - Avg loss: 4.110740, Accuracy: 0.003332, Non-Pad-Accuracy: 0.005613


wandb: Wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1 completed!
Train Results        - Avg loss: 0.529367, Accuracy: 0.891165, Non-Pad-Accuracy: 0.848086
Validation Results   - Avg loss: 0.270159, Accuracy: 0.936748, Non-Pad-Accuracy: 0.885670
Epoch 2 completed!
Train Results        - Avg loss: 0.242492, Accuracy: 0.941147, Non-Pad-Accuracy: 0.890211
Validation Results   - Avg loss: 0.223068, Accuracy: 0.943551, Non-Pad-Accuracy: 0.893895


In [ ]:
best_model_checkpoint = list(checkpoint_dir.glob('best_model_*'))
assert(len(best_model_checkpoint) == 1)
best_model_checkpoint = best_model_checkpoint[0]

files.download(best_model_checkpoint) 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "{log_dir}"

zip and download logs

In [ ]:
log_zip =  output_dir / 'logs' / (run_name + '_logs.zip')
!zip -r "$log_zip" "$log_dir"
files.download(log_zip) 

  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/ (stored 0%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/train/ (stored 0%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/train/events.out.tfevents.1586905646.00b82a3f534b.127.9 (deflated 94%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/validation/ (stored 0%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/validation/events.out.tfevents.1586905646.00b82a3f534b.127.10 (deflated 73%)


# Prediction exploration

TODO:
- decode class numbers

In [ ]:
batch = next(iter(val_loader))

In [ ]:
x, y = prepare_batch_lstm(batch)

In [ ]:
lstm_encoder.train()

In [ ]:
y.argmax(dim=2)[2]

In [ ]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2)[2]

In [ ]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2).unique()